In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb

from sklearn.metrics import mean_squared_error, r2_score
import time

In [3]:
X_data = pd.read_csv('final_X_data_hause_pr.csv')
y_data = pd.read_csv('final_y_data_hause_pr.csv')

In [5]:
X_data

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,KitchenQual_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf
0,65,8450,7,5,2003,2003,196,706,150,856,...,0,0,1,0,0,0,0,0,1,0
1,80,9600,6,8,1976,1976,0,978,284,1262,...,1,0,1,0,0,0,0,0,1,0
2,68,11250,7,5,2001,2002,162,486,434,920,...,0,0,1,0,0,0,0,0,1,0
3,60,9550,7,5,1915,1970,0,216,540,756,...,0,0,0,0,0,0,1,0,0,1
4,84,14260,8,5,2000,2000,350,655,490,1145,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62,7917,6,5,1999,2000,0,0,953,953,...,1,0,1,0,0,0,0,0,1,0
1456,85,13175,6,6,1978,1988,119,790,589,1542,...,1,0,1,0,0,0,0,0,0,1
1457,66,9042,7,9,1941,2006,0,275,877,1152,...,0,0,1,0,0,0,0,0,1,0
1458,68,9717,5,6,1950,1996,0,49,0,1078,...,0,0,1,0,0,0,0,0,0,1


In [4]:
# pip install xgboost

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [6]:
model_xgb = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-2
)

In [7]:
start_time = time.time()
model_xgb.fit(X_train, y_train)
end_time = time.time()
time_fit_cpu = end_time - start_time

In [8]:
model_xgb.score(X_test, y_test)

0.9160538911819458

In [9]:
model_xgb.score(X_train, y_train)

0.9639922380447388

In [10]:
y_pred = model_xgb.predict(X_test)

In [11]:
round((mean_squared_error(y_test, y_pred)) ** 0.5, 2)

25375.07

## Теперь с подбором гиперпараметров

In [13]:
model_xgb2 = xgb.XGBRegressor(random_state=42)

In [14]:
parametrs = {'n_estimators': [100, 200, 300],
            'max_depth': [3, 5],
            'learning_rate': [0.1],
            'reg_lambda': [0, 1, 2],
            'gamma': [0, 1, 2]}

In [15]:
grid_search_cv_xgb = GridSearchCV(model_xgb2, parametrs, cv=5, n_jobs=-2)

In [16]:
start_time = time.time()
grid_search_cv_xgb.fit(X_train, y_train)
end_time = time.time()
time_fit = end_time - start_time

In [17]:
print(f'время обучения модели {round(time_fit, 2)}с')

время обучения модели 117.92с


In [18]:
grid_search_cv_xgb.best_params_

{'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 300,
 'reg_lambda': 1}

In [19]:
best_model_xgb = grid_search_cv_xgb.best_estimator_

In [20]:
round(best_model_xgb.score(X_test, y_test), 4)  #r2_score

0.9188